In [1]:
# Librerias generales
import pandas as pd
import numpy as np
import json

# Time
import datetime as dt
from datetime import datetime, timedelta

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
from dython.model_utils import roc_graph
from dython.nominal import associations
%matplotlib inline

# OneHotEncoding
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2 # for chi-squared feature selection
import prince

from sklearn import metrics

import os
import json
import joblib

from pandas.io.json import json_normalize
from datetime import datetime 
from tqdm import tqdm
from sklearn.preprocessing import PowerTransformer
import umap
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy import stats

from sklearn.cluster import KMeans
from kmodes.kprototypes import KPrototypes


from lightgbm import LGBMClassifier
import shap
from sklearn.model_selection import cross_val_score


# Algoritmos
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import preprocessing 
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

# Pipeline
from sklearn.pipeline import make_pipeline

from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# Configuración de pandas 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# new_data = Processing_data(data)
# new_data_cat = Processing_data(data_cath)
# Merge manual 

In [195]:
# 1. Leo data numerica
# 2. Leo data categorica
# 3. new_data = Processing_data(data, cath_data)
# 4. Ejecuta - merge cuando yo le diga

In [ ]:
# Resultado

In [8]:
class Processing_data():
    """
    Processing data steps:
    1- After reading the file (excel or csv) fill n/a rows of the file on json format rows using -fill_na_json method-
    2- Clean the dataframe to have the first draft of you desired output using -clean_data method-
    3- Delete unnecessary columns on numerical data using -delete_unnecessary_columns method-
    4- Organize the dataframe to have you desired dataframe to study using -organizing_data-
    """

    def __init__(self, data, cat_data):
        """Constructor of attributes"""
        self.data = data
        self.kproto = []
        self.clusters = []
        self.cat_data = cat_data

    def fill_na_json(self):
        """Filling missing values in json format columns"""
        col_names = ['expected_detail', 'paid_detail']
        for col_name in col_names:
            self.data[col_name] = self.data[col_name].apply(
                lambda x: "{}" if pd.isna(x) else x)
        return self.data

    @staticmethod
    def normalize_data(df, col_name):
        data = df[col_name].apply(json.loads)
        return pd.json_normalize(data)

    def clean_data(self):
        """
        Cleaning process of data:
        1- normalize
        2- deleting useless state (GRACE, PENDING, AND PARTIALLY PAID) on data
        """
        expected = Processing_data.normalize_data(self.data, 'expected_detail')
        paid = Processing_data.normalize_data(self.data, 'paid_detail')
        self.data = self.data.join(expected).join(paid, rsuffix='_paid')
        self.data = self.data[self.data.state != "GRACE"]
        self.data = self.data[self.data.state != "PENDING"]
        self.data = self.data[self.data.state != "PARTIALLY_PAID"]
        return self.data

#    def normalize_expected(self):
#        """Normalize json format in expected_detail and adding it to the df"""
#        info = self.data['expected_detail'].apply(json.loads)
#        expected = pd.json_normalize(info)
#        self.data = self.data.join(expected)
#
#    def normalize_paid(self):
#        """Normalize json format in paid_detail and adding it to the df"""
#        info = self.data['paid_detail'].apply(json.loads)
#        paid = pd.json_normalize(info)
#        self.data = self.data.join(paid, rsuffix='_paid')
#
#    def deleting_ocurrencys_state(self):
#        """Deletes a rows with state grace, pending and partially_paid"""
#        self.data = self.data[self.data.state != "GRACE"]
#        self.data = self.data[self.data.state != "PENDING"]
#        self.data = self.data[self.data.state != "PARTIALLY_PAID"]
#        return self.data

    def delete_unnecessary_columns(self):
        """Deletes unnecesary columns produced by the json format"""
        data_droped_columns = ['penalty', 'interest', 'insurance', 'principal', 'taxRetentionValue',
                               'taxRetentionPercentage', 'legalCharge', 'preLegalCharge', 'feesPaid',
                               'fngTotal', 'fngValue',
                               'fngPercent', 'fngVatValue', 'fngVatPercent', 'monthlyDeferredInterest',
                               'penalty_paid', 'interest_paid', 'insurance_paid',
                               'principal_paid', 'taxRetentionValue_paid',
                               'taxRetentionPercentage_paid', 'legalCharge_paid',
                               'preLegalCharge_paid', 'feesPaid_paid', 'fngTotal_paid',
                               'fngValue_paid', 'fngPercent_paid', 'fngVatValue_paid',
                               'fngVatPercent_paid', 'monthlyDeferredInterest_paid', 'expected_detail', 'paid_detail']
        self.data = self.data.drop(columns=data_droped_columns)
        return self.data

    def organizing_data(self):
        """
        Organize data after having the first draft of your dataframe and fills total na rows with 0
        and groups clients by their six newest loan_id
        This is step three
        """
        self.data = self.data.sort_values(
            ['client_id', 'loan_id', 'expected_date'])
        self.data['total'] = self.data['total'].fillna(0)
        self.data['total_paid'] = self.data['total_paid'].fillna(0)
        self.data = self.data.groupby('loan_id').filter(
            lambda x: x['loan_id'].value_counts() > 6).groupby('loan_id').tail(6)
        return self.data

    def checking_cat_data(self):
        """Checks for the clients that are present in the two tables"""
        self.cat_data = self.cat_data.sort_values(
            by='ID Cliente', ascending=True)

        rep_columns = ['Procesos judiciales', 'Alertas', 'Website empresa',
                       'Instagram empresa', 'LinkedIn empresa',
                       'LinkedIn empresarios', 'Impacto', 'Acceso previso a la banca',
                       'Mujeres empresarias']

        # Replace values for Si/No
        self.cat_data[rep_columns] = self.cat_data[rep_columns].replace({'No procesos judiciales': 'No',
                                                                         'Sí procesos judiciales': 'Si',
                                                                         'No Alertas': 'No',
                                                                         'Sí Alertas': 'Si',
                                                                         'No website': 'No',
                                                                         'Si website': 'Si',
                                                                         'No Ig': 'No',
                                                                         'Si Ig': 'Si',
                                                                         'No LinkedIn': 'No',
                                                                         'Si LinkedIn': 'Si',
                                                                         'No LinkedIn empresarios': 'No',
                                                                         'Si LinkedIn empresarios': 'Si',
                                                                         'Si Impacto': 'Si',
                                                                         'No Impacto': 'No',
                                                                         'Si acceso a la banca': 'Si',
                                                                         'No acceso a la banca': 'No',
                                                                         'No mujeres empresarias': 'No',
                                                                         'Si mujeres empresarias': 'Si'})
        return self.cat_data

    def deleting_cat_info(self):
        """Deletes cat_data to analize clients on mean"""
        cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
                    'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
                    'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
                    'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
                    'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
                    'Impacto', 'Acceso previso a la banca', '# Empleados',
                    'Mujeres empresarias', 'Mujeres en cargos directivos']

        self.data = self.data.drop(columns=cat_cols)
        return self.data

    def merge_two_tables(self):
        """Merging two tables: Numerical and cathegorical"""
        self.data = self.data.merge(
            self.cat_data, left_on='client_id', right_on='ID Cliente')
        return self.data

    def customer_level(self):
        """Compress dataframe into a better study"""
        self.data = self.data.groupby(['client_id', 'loan_id']).agg({
            'state': lambda x: x.iloc[-1],  # Devuelve el último state
            'arrears_days': 'sum',
            'total_paid': 'sum'}).round(0)  # Se suman todas las facturas
        self.data.reset_index(inplace=True)
        return self.data

    def head(self):
        """print head of df"""
        return self.data.head()

    # Funcion compacta de limpieza de data
    def data_cleaning(self):
        """This function resumes all the steps of organizing and cleaning the datasets"""
        self.data = self.fill_na_json()
        self.data = self.clean_data()
        self.data = self.delete_unnecessary_columns()
        self.data = self.organizing_data()
        self.cat_data = self.checking_cat_data()
        self.data = self.merge_two_tables()
        self.data = self.deleting_cat_info()
        self.data = self.customer_level()
        self.data = self.merge_two_tables()
        return self.data
        

#    def organizing_cath_data:
#        """Organizing cathegorical data"""
#        cat_data = cat_data.sort_values(by='ID Cliente', ascending=True)
# ---------------------------------------

    def transform_data(self):
        """transform_data : Transforma los datos numericos con el metodo PowerTransformer"""
        self.data = self.data_cleaning()
        #columns = ['client_id', 'loan_id']
        # Si el dataframe tiene las columnas borrarlo
        #if all(item in list(self.data.columns) for item in columns):
        #    self.data.drop(columns, axis=1, inplace=True)

        # Transformar la data
        for c in self.data.select_dtypes(exclude='object').columns:
            pt = PowerTransformer()
            self.data[c] = pt.fit_transform(
                np.array(self.data[c]).reshape(-1, 1))
        return self.data

    def reduction_dim(self):
        """reduction: Reduce la dimensionalidad de los datos aplicando Analisis Factorial de Datos Mixtos(FAMD)"""
        #self.data = self.transform_data()
        self.data = self.transform_data()
        
        self.data['state'] = self.data['state'].replace(to_replace="LATE",value="0")
        self.data['state'] = self.data['state'].replace(to_replace="PAID",value="1")
        self.data['state'] = self.data['state'].astype(object)
        #self.data
        #self.data.reset_index(drop=True, inplace=True)

        # Declarar metodo para aplicar FAMD
        famd = prince.FAMD(
            n_components=2,
            n_iter=3,
            copy=True,
            check_input=True,
            engine='auto',
            random_state=42)

        # Ajustar y transformar la dimensión aplicando FAMD
        famd = famd.fit(self.data)
        transformada = famd.transform(self.data)

        Y = transformada.to_numpy()
        principalDf_famd = pd.DataFrame(
            data=Y, columns=['principal component 1', 'principal component 2'])
        finalDf_Cat_famd = pd.concat(
            [principalDf_famd, self.data['state']], axis=1, ignore_index=True)

        self.data = finalDf_Cat_famd
        return self.data

    def execute_model(self):
        """execute: Función que ejecuta el modelo con los datos procesados"""
        self.data = self.reduction_dim().to_numpy()
        self.kproto = KPrototypes(n_clusters=2, init='Huang', verbose=0,
                                  random_state=42, max_iter=20, n_init=50, n_jobs=-20, gamma=.25)
        self.clusters = self.kproto.fit_predict(self.data, categorical=[2])
        #self.kproto = kproto
        print(self.kproto)
        #print(self.clusters)
        return self.kproto

    def guardar_model(self):
        joblib.dump(self.execute_model(), 'testeo.joblib')
        print("Se ha guardado correctamente!")

In [3]:
#------------------------------------------------------------------

In [15]:
path_file = "loan_schedules_2.xlsx"
cols = ['client_id',
        'loan_schedule_id',
        'loan_id',
        'index',
        'expected_date',
        'state',
        'arrears_days',
        'expected_detail',
        'paid_detail']
data = pd.read_excel(path_file, sheet_name = 0, header = 0, usecols = cols).replace('NaT', np.nan)

In [16]:
path_file = "data_sempli_holberton.xlsx"
cat_cols = ['ID Cliente', 'Monto Acumulado', 'Uso de los recursos', 'Plazo',
       'Sector', 'Ingresos', 'Ubicación', 'Estrato Mínimo',
       'Procesos judiciales', 'Alertas', 'Score Bureau Empresa', 'Huellas de Consulta', 'Tiempo en el negocio',
       'Website empresa', 'Instagram empresa', 'LinkedIn empresa',
       'LinkedIn empresarios', 'Edad empresarios', 'Activador', 'Número de accionistas',
       'Impacto', 'Acceso previso a la banca', '# Empleados',
       'Mujeres empresarias', 'Mujeres en cargos directivos']
cat_data = pd.read_excel(path_file, sheet_name = 0, header = 2, usecols = cat_cols).replace('SIN INFO', np.nan)
cat_data = cat_data.dropna()

In [27]:
new_data = Processing_data(data, cat_data)
type(new_data)

__main__.Processing_data

In [18]:
%%time
new_data.execute_model()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


KPrototypes(gamma=0.25, max_iter=20, n_clusters=2, n_init=50, n_jobs=-20,
            random_state=42)
Wall time: 40.9 s


KPrototypes(gamma=0.25, max_iter=20, n_clusters=2, n_init=50, n_jobs=-20,
            random_state=42)

In [21]:
%%time
new_data.guardar_model()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


KPrototypes(gamma=0.25, max_iter=20, n_clusters=2, n_init=50, n_jobs=-20,
            random_state=42)
Se ha guardado correctamente!


In [23]:
data_prueba = new_data.data_cleaning()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 577 entries, 0 to 576
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   client_id                     577 non-null    int64  
 1   loan_id                       577 non-null    int64  
 2   state                         577 non-null    object 
 3   arrears_days                  577 non-null    int64  
 4   total_paid                    577 non-null    float64
 5   ID Cliente                    577 non-null    float64
 6   Monto Acumulado               577 non-null    int64  
 7   Uso de los recursos           577 non-null    object 
 8   Plazo                         577 non-null    object 
 9   Sector                        577 non-null    object 
 10  Ingresos                      577 non-null    object 
 11  Ubicación                     577 non-null    object 
 12  Estrato Mínimo                577 non-null    int64  
 13  Proce

#### Testeando modelo

In [24]:
clf_model = joblib.load('testeo.joblib')

In [25]:
# Generando un muestra aleatoria de 20 clientes
prueba= data_prueba.sample(7)
numerica = prueba.select_dtypes(exclude='object')
categorica = prueba.select_dtypes(exclude=['int64', 'float64'])

In [26]:
#Procesar los datos de prueba
data_modelo = Processing_data(numerica, categorica)
entrenar = data_modelo.reduction_dim().to_numpy()

KeyError: 'expected_detail'

In [119]:
print(f'clientes a evaluar {len(entrenar)}\n')
for i in range(len(entrenar)):
    print(f'cliente #{i} pertence al cluster: {clf_model.predict(entrenar, categorical=[2])[i]}')
    

clientes a evaluar 577

cliente #0 pertence al cluster: 0
cliente #1 pertence al cluster: 0
cliente #2 pertence al cluster: 1
cliente #3 pertence al cluster: 0
cliente #4 pertence al cluster: 0
cliente #5 pertence al cluster: 1
cliente #6 pertence al cluster: 1
cliente #7 pertence al cluster: 0
cliente #8 pertence al cluster: 0
cliente #9 pertence al cluster: 0
cliente #10 pertence al cluster: 0
cliente #11 pertence al cluster: 0
cliente #12 pertence al cluster: 0
cliente #13 pertence al cluster: 0
cliente #14 pertence al cluster: 0
cliente #15 pertence al cluster: 0
cliente #16 pertence al cluster: 0
cliente #17 pertence al cluster: 0
cliente #18 pertence al cluster: 0
cliente #19 pertence al cluster: 0
cliente #20 pertence al cluster: 0
cliente #21 pertence al cluster: 0
cliente #22 pertence al cluster: 0
cliente #23 pertence al cluster: 0
cliente #24 pertence al cluster: 0
cliente #25 pertence al cluster: 1
cliente #26 pertence al cluster: 1
cliente #27 pertence al cluster: 0
client

cliente #229 pertence al cluster: 0
cliente #230 pertence al cluster: 0
cliente #231 pertence al cluster: 0
cliente #232 pertence al cluster: 0
cliente #233 pertence al cluster: 0
cliente #234 pertence al cluster: 0
cliente #235 pertence al cluster: 0
cliente #236 pertence al cluster: 0
cliente #237 pertence al cluster: 0
cliente #238 pertence al cluster: 1
cliente #239 pertence al cluster: 0
cliente #240 pertence al cluster: 1
cliente #241 pertence al cluster: 0
cliente #242 pertence al cluster: 1
cliente #243 pertence al cluster: 1
cliente #244 pertence al cluster: 1
cliente #245 pertence al cluster: 1
cliente #246 pertence al cluster: 0
cliente #247 pertence al cluster: 1
cliente #248 pertence al cluster: 1
cliente #249 pertence al cluster: 1
cliente #250 pertence al cluster: 1
cliente #251 pertence al cluster: 0
cliente #252 pertence al cluster: 1
cliente #253 pertence al cluster: 1
cliente #254 pertence al cluster: 0
cliente #255 pertence al cluster: 1
cliente #256 pertence al clu

cliente #459 pertence al cluster: 1
cliente #460 pertence al cluster: 1
cliente #461 pertence al cluster: 1
cliente #462 pertence al cluster: 1
cliente #463 pertence al cluster: 1
cliente #464 pertence al cluster: 1
cliente #465 pertence al cluster: 1
cliente #466 pertence al cluster: 1
cliente #467 pertence al cluster: 1
cliente #468 pertence al cluster: 1
cliente #469 pertence al cluster: 1
cliente #470 pertence al cluster: 1
cliente #471 pertence al cluster: 1
cliente #472 pertence al cluster: 0
cliente #473 pertence al cluster: 0
cliente #474 pertence al cluster: 1
cliente #475 pertence al cluster: 1
cliente #476 pertence al cluster: 1
cliente #477 pertence al cluster: 0
cliente #478 pertence al cluster: 1
cliente #479 pertence al cluster: 1
cliente #480 pertence al cluster: 1
cliente #481 pertence al cluster: 1
cliente #482 pertence al cluster: 1
cliente #483 pertence al cluster: 1
cliente #484 pertence al cluster: 1
cliente #485 pertence al cluster: 1
cliente #486 pertence al clu

In [8]:
new_data.transform_data()

overflow encountered in power
overflow encountered in multiply
overflow encountered in reduce


,state,arrears_days,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
0,0.745155,0.068201,1.990516,-2.323541,0.247658,CR,25 a 36 meses,Servicios,Seed,Medellín,0.481027,No,No,0.592443,0.514087,0.824309,Si,Si,Si,Si,0.542674,Internet,0.017484,No,Si,0.176367,No,0.447494
1,0.745155,0.219112,1.308246,-2.323541,0.247658,CR,25 a 36 meses,Servicios,Seed,Medellín,0.481027,No,No,0.592443,0.514087,0.824309,Si,Si,Si,Si,0.542674,Internet,0.017484,No,Si,0.176367,No,0.447494
2,0.745155,-0.027869,-0.515458,-2.033775,-1.405435,KT - EX,13 a 24 meses,Servicios,Seed,Medellín,-1.519614,No,No,0.585245,-0.688637,0.456239,Si,No,Si,No,-0.942694,SocialNetworks,0.736474,No,Si,-1.700640,No,-1.459134
3,0.745155,0.735415,-0.901116,-1.901462,-1.626569,SP,13 a 24 meses,Industria,Seed,Medellín,-0.523889,No,No,0.145027,-1.130869,0.638175,Si,No,Si,Si,0.451404,CreditProfessionalContact,0.017484,No,Si,-1.107593,No,-1.459134
4,0.745155,0.735415,-0.946665,-1.901462,-1.626569,SP,13 a 24 meses,Industria,Seed,Medellín,-0.523889,No,No,0.145027,-1.130869,0.638175,Si,No,Si,Si,0.451404,CreditProfessionalContact,0.017484,No,Si,-1.107593,No,-1.459134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,-1.342003,0.335600,-0.475159,1.951817,-1.405435,KT - CO,13 a 24 meses,Servicios,Seed,Bogotá D.C.,0.481027,No,No,0.900597,-0.688637,1.014737,Si,Si,No,Si,-0.054151,SocialNetworks,0.017484,No,Si,-0.896798,No,-1.459134
573,-1.342003,-0.613883,0.238679,2.032265,0.012235,KT - EX,25 a 36 meses,Servicios,Venture,Bogotá D.C.,-0.523889,No,Si,0.358389,0.069432,0.278411,Si,No,Si,Si,-0.166537,CommunicationMedia,-1.311581,No,Si,0.295523,No,0.867677
574,-1.342003,0.676864,-0.151688,2.040592,-0.436514,CR,25 a 36 meses,Servicios,Venture,Bogotá D.C.,-0.523889,No,No,0.492277,-0.688637,0.824309,No,No,No,No,-0.530803,Internet,0.736474,Si,Si,0.038432,No,1.411840
575,0.745155,-1.494842,1.209378,2.084151,0.750264,KT - EX,25 a 36 meses,Servicios,Venture,Medellín,1.493650,No,Si,0.556522,-0.688637,-0.065288,Si,Si,No,Si,-0.166537,SocialNetworks,0.017484,Si,Si,-0.124416,No,2.024831


In [34]:
%%time
new_data.execute_model()

TypeError: invalid type promotion

In [16]:
new_data.data_cleaning()

KeyError: 'expected_detail'

In [271]:
new_data.head()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,expected_detail,paid_detail
0,1471,952,54,7,2019-02-27,PENDING,0,"{""total"": 5197196.73, ""penalty"": 0, ""interest""...","{""total"": 0, ""penalty"": 0, ""interest"": 0, ""ins..."
1,2442,13696,592,6,2019-03-18,PAID,0,"{""total"": 22872996.50, ""penalty"": 0, ""interest...","{""total"": 22872996.50, ""penalty"": 0, ""interest..."
2,9273,43776,1831,3,2020-03-04,PAID,5,"{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""...","{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""..."
3,2952,25418,1065,2,2019-02-12,PAID,0,"{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""...","{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""..."
4,2459,14548,628,13,2019-11-19,LATE,76,{},{}


In [272]:
new_data.fill_na_json()
print(new_data)

In [273]:
new_data.head()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,expected_detail,paid_detail
0,1471,952,54,7,2019-02-27,PENDING,0,"{""total"": 5197196.73, ""penalty"": 0, ""interest""...","{""total"": 0, ""penalty"": 0, ""interest"": 0, ""ins..."
1,2442,13696,592,6,2019-03-18,PAID,0,"{""total"": 22872996.50, ""penalty"": 0, ""interest...","{""total"": 22872996.50, ""penalty"": 0, ""interest..."
2,9273,43776,1831,3,2020-03-04,PAID,5,"{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""...","{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""..."
3,2952,25418,1065,2,2019-02-12,PAID,0,"{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""...","{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""..."
4,2459,14548,628,13,2019-11-19,LATE,76,{},{}


In [274]:
print(type(new_data))

<class '__main__.Processing_data'>


In [275]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [276]:
new_data.clean_data()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,expected_detail,paid_detail,total,penalty,interest,insurance,principal,taxRetentionValue,taxRetentionPercentage,fngTotal,fngValue,fngPercent,fngVatValue,legalCharge,fngVatPercent,preLegalCharge,feesPaid,monthlyDeferredInterest,total_paid,penalty_paid,interest_paid,insurance_paid,principal_paid,taxRetentionValue_paid,taxRetentionPercentage_paid,fngTotal_paid,fngValue_paid,fngPercent_paid,fngVatValue_paid,legalCharge_paid,fngVatPercent_paid,preLegalCharge_paid,feesPaid_paid,monthlyDeferredInterest_paid
1,2442,13696,592,6,2019-03-18,PAID,0,"{""total"": 22872996.50, ""penalty"": 0, ""interest...","{""total"": 22872996.50, ""penalty"": 0, ""interest...",22872996.50,0.0,358105.48,0.0,22514891.02,14324.2192,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22872996.50,0.0,358105.48,0.0,22514891.02,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9273,43776,1831,3,2020-03-04,PAID,5,"{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""...","{""total"": 7322168.42, ""penalty"": 0, ""fngTotal""...",7322168.42,0.0,3015180.80,59878.0,4306987.62,120607.2320,0.04,0.0,0.0,0.028,0.0,NaN,0.19,NaN,NaN,NaN,7322168.42,0.0,3015180.80,0.0,4306987.62,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2952,25418,1065,2,2019-02-12,PAID,0,"{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""...","{""total"": 4264953.71, ""penalty"": 0, ""feesPaid""...",4264953.71,0.0,1519955.30,29939.0,2744998.41,60798.2120,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4264953.71,0.0,1519955.30,0.0,2744998.41,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2459,14548,628,13,2019-11-19,LATE,76,{},{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4458,31214,1320,6,2019-11-27,PAID,0,"{""total"": 2598166.29, ""penalty"": 0, ""feesPaid""...","{""total"": 2598166.29, ""penalty"": 0, ""feesPaid""...",2598166.29,0.0,493134.76,10828.0,2105031.53,19725.3904,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2598166.29,0.0,493134.76,0.0,2105031.53,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12754,7450,38103,1609,5,2020-02-13,PAID,0,"{""total"": 1656810.57, ""penalty"": 0, ""feesPaid""...","{""total"": 1656810.57, ""penalty"": 0, ""feesPaid""...",1656810.57,0.0,663022.29,0.0,993788.28,0.0000,0.04,0.0,0.0,0.000,0.0,NaN,0.00,NaN,NaN,0.0,1656810.57,0.0,663022.29,0.0,993788.28,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
12755,7450,38104,1609,6,2020-03-13,PAID,0,"{""total"": 1656810.57, ""penalty"": 0, ""feesPaid""...","{""total"": 1656810.57, ""penalty"": 0, ""feesPaid""...",1656810.57,0.0,647022.30,0.0,1009788.27,0.0000,0.04,0.0,0.0,0.000,0.0,NaN,0.00,NaN,NaN,0.0,1656810.57,0.0,647022.30,0.0,1009788.27,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
12756,1068,24104,984,21,2020-03-09,LATE,22,{},{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12758,2454,15828,682,10,2019-08-08,PAID,0,"{""total"": 2772721.57, ""penalty"": 0, ""interest""...","{""total"": 2772721.57, ""penalty"": 0, ""interest""...",2772721.57,0.0,131385.57,0.0,2641336.00,5255.4228,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2772721.57,0.0,131385.57,0.0,2641336.00,0.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [277]:
#new_data.deleting_ocurrencys_state()
#new_data.head()

In [278]:
#new_data.normalize_expected()
#new_data.normalize_paid()
#new_data.head()

In [279]:
new_data.delete_unnecessary_columns()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,total,total_paid
1,2442,13696,592,6,2019-03-18,PAID,0,22872996.50,22872996.50
2,9273,43776,1831,3,2020-03-04,PAID,5,7322168.42,7322168.42
3,2952,25418,1065,2,2019-02-12,PAID,0,4264953.71,4264953.71
4,2459,14548,628,13,2019-11-19,LATE,76,NaN,NaN
5,4458,31214,1320,6,2019-11-27,PAID,0,2598166.29,2598166.29
...,...,...,...,...,...,...,...,...,...
12754,7450,38103,1609,5,2020-02-13,PAID,0,1656810.57,1656810.57
12755,7450,38104,1609,6,2020-03-13,PAID,0,1656810.57,1656810.57
12756,1068,24104,984,21,2020-03-09,LATE,22,NaN,NaN
12758,2454,15828,682,10,2019-08-08,PAID,0,2772721.57,2772721.57


In [280]:
new_data.organizing_data()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,total,total_paid
5677,14,80,14,6,2017-10-27,PAID,4,1497723.06,1497723.06
860,14,81,14,7,2017-11-27,PAID,0,1499416.03,1499416.03
861,14,82,14,8,2017-12-27,PAID,0,1497723.06,1497723.06
862,14,83,14,9,2018-01-29,PAID,0,1497723.06,1497723.06
863,14,84,14,10,2018-02-27,PAID,0,20078872.73,20078872.73
...,...,...,...,...,...,...,...,...,...
6763,7279,37729,1595,3,2019-12-02,PAID,0,3233354.83,3233354.83
6783,7279,37730,1595,4,2019-12-30,PAID,0,3233354.83,3233354.83
6786,7279,37731,1595,5,2020-01-30,PAID,0,3233354.83,3233354.83
6787,7279,37732,1595,6,2020-03-02,PAID,0,3233354.83,3233354.83


In [281]:
new_data.checking_cat_data()
new_data.merge_two_tables()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,total,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
0,241,25718,1076,8,2019-08-14,PAID,16,5947504.88,5947504.88,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
1,241,25719,1076,9,2019-09-16,PAID,1,5972929.73,5972929.73,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
2,241,25720,1076,10,2019-10-30,PAID,0,5941645.15,5941645.15,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
3,241,25721,1076,11,2019-11-29,PAID,0,5939653.40,5939653.40,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
4,241,25722,1076,12,2019-12-30,PAID,0,5939653.40,5939653.40,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3457,7279,37729,1595,3,2019-12-02,PAID,0,3233354.83,3233354.83,7279.0,50000000,KT - CO,13 a 24 meses,Servicios,Venture,Bogotá D.C.,3,No,No,226.0,8.0,1990,Si,No,No,Si,56,Referrer,2.0,No,Si,6,No,0
3458,7279,37730,1595,4,2019-12-30,PAID,0,3233354.83,3233354.83,7279.0,50000000,KT - CO,13 a 24 meses,Servicios,Venture,Bogotá D.C.,3,No,No,226.0,8.0,1990,Si,No,No,Si,56,Referrer,2.0,No,Si,6,No,0
3459,7279,37731,1595,5,2020-01-30,PAID,0,3233354.83,3233354.83,7279.0,50000000,KT - CO,13 a 24 meses,Servicios,Venture,Bogotá D.C.,3,No,No,226.0,8.0,1990,Si,No,No,Si,56,Referrer,2.0,No,Si,6,No,0
3460,7279,37732,1595,6,2020-03-02,PAID,0,3233354.83,3233354.83,7279.0,50000000,KT - CO,13 a 24 meses,Servicios,Venture,Bogotá D.C.,3,No,No,226.0,8.0,1990,Si,No,No,Si,56,Referrer,2.0,No,Si,6,No,0


In [282]:
new_data.deleting_cat_info()

,client_id,loan_schedule_id,loan_id,index,expected_date,state,arrears_days,total,total_paid
0,241,25718,1076,8,2019-08-14,PAID,16,5947504.88,5947504.88
1,241,25719,1076,9,2019-09-16,PAID,1,5972929.73,5972929.73
2,241,25720,1076,10,2019-10-30,PAID,0,5941645.15,5941645.15
3,241,25721,1076,11,2019-11-29,PAID,0,5939653.40,5939653.40
4,241,25722,1076,12,2019-12-30,PAID,0,5939653.40,5939653.40
...,...,...,...,...,...,...,...,...,...
3457,7279,37729,1595,3,2019-12-02,PAID,0,3233354.83,3233354.83
3458,7279,37730,1595,4,2019-12-30,PAID,0,3233354.83,3233354.83
3459,7279,37731,1595,5,2020-01-30,PAID,0,3233354.83,3233354.83
3460,7279,37732,1595,6,2020-03-02,PAID,0,3233354.83,3233354.83


In [283]:
new_data.customer_level()

,client_id,loan_id,state,arrears_days,total_paid
0,241,1076,PAID,17,147915805.0
1,241,1244,PAID,23,91852726.0
2,362,1469,PAID,14,15211138.0
3,428,119,PAID,64,8713879.0
4,428,120,PAID,64,8102608.0
...,...,...,...,...,...
572,6734,1499,LATE,29,16038476.0
573,6982,1503,LATE,4,36277437.0
574,7008,1593,LATE,57,23829284.0
575,7145,1492,PAID,0,85184813.0


In [284]:
new_data.merge_two_tables()

,client_id,loan_id,state,arrears_days,total_paid,ID Cliente,Monto Acumulado,Uso de los recursos,Plazo,Sector,Ingresos,Ubicación,Estrato Mínimo,Procesos judiciales,Alertas,Score Bureau Empresa,Huellas de Consulta,Tiempo en el negocio,Website empresa,Instagram empresa,LinkedIn empresa,LinkedIn empresarios,Edad empresarios,Activador,Número de accionistas,Impacto,Acceso previso a la banca,# Empleados,Mujeres empresarias,Mujeres en cargos directivos
0,241,1076,PAID,17,147915805.0,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
1,241,1244,PAID,23,91852726.0,241.0,250000000,CR,25 a 36 meses,Servicios,Seed,Medellín,5,No,No,925.0,8.0,2016,Si,Si,Si,Si,45,Internet,2.0,No,Si,16,No,2
2,362,1469,PAID,14,15211138.0,362.0,50000000,KT - EX,13 a 24 meses,Servicios,Seed,Medellín,3,No,No,924.0,2.0,2014,Si,No,Si,No,32,SocialNetworks,3.0,No,Si,3,No,0
3,428,119,PAID,64,8713879.0,428.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861.0,1.0,2015,Si,No,Si,Si,44,CreditProfessionalContact,2.0,No,Si,5,No,0
4,428,120,PAID,64,8102608.0,428.0,40000000,SP,13 a 24 meses,Industria,Seed,Medellín,4,No,No,861.0,1.0,2015,Si,No,Si,Si,44,CreditProfessionalContact,2.0,No,Si,5,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,6734,1499,LATE,29,16038476.0,6734.0,50000000,KT - CO,13 a 24 meses,Servicios,Seed,Bogotá D.C.,5,No,No,967.0,2.0,2017,Si,Si,No,Si,39,SocialNetworks,2.0,No,Si,6,No,0
573,6982,1503,LATE,4,36277437.0,6982.0,200000000,KT - EX,25 a 36 meses,Servicios,Venture,Bogotá D.C.,4,No,Si,892.0,5.0,2013,Si,No,Si,Si,38,CommunicationMedia,1.0,No,Si,18,No,3
574,7008,1593,LATE,57,23829284.0,7008.0,130000000,CR,25 a 36 meses,Servicios,Venture,Bogotá D.C.,4,No,No,911.0,2.0,2016,No,No,No,No,35,Internet,3.0,Si,Si,14,No,5
575,7145,1492,PAID,0,85184813.0,7145.0,400000000,KT - EX,25 a 36 meses,Servicios,Venture,Medellín,6,No,Si,920.0,2.0,2011,Si,Si,No,Si,38,SocialNetworks,2.0,Si,Si,12,No,9
